## Step 1: Install Required Packages

In [ ]:
%%capture
!pip install youtube-transcript-api langchain langchain-openai langchain-community langchain-text-splitters langchain-chroma chromadb openai gradio torch
print("✅ All packages installed successfully!")

## Step 2: Set Your OpenAI API Key

Get your API key from: https://platform.openai.com/api-keys

In [ ]:
import os
from google.colab import userdata

# Option 1: Use Colab Secrets (Recommended)
# Add your key in Colab: Secrets icon (🔑) → Add "OPENAI_API_KEY"
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except:
    # Option 2: Enter manually (less secure)
    OPENAI_API_KEY = input("Enter your OpenAI API key: ")
    print("✅ API key entered manually")

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Step 3: Import Libraries and Setup

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_core.documents import Document
import json

print("✅ Libraries imported successfully")

## Step 4: Transcript Fetcher Class

In [ ]:
class YouTubeTranscriptFetcher:
    """Fetches YouTube video transcripts"""
    
    @staticmethod
    def extract_video_id(url: str) -> str:
        """Extract video ID from YouTube URL"""
        if "youtube.com" in url or "youtu.be" in url:
            if "v=" in url:
                return url.split("v=")[1].split("&")[0]
            elif "youtu.be/" in url:
                return url.split("youtu.be/")[1].split("?")[0]
        return url  # Already a video ID
    
    def fetch_transcript(self, video_id: str) -> dict:
        """Fetch transcript for a single video"""
        video_id = self.extract_video_id(video_id)
        
        try:
            # Get transcript
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            
            # Combine all text
            full_text = " ".join([entry['text'] for entry in transcript_list])
            
            return {
                'video_id': video_id,
                'transcript': full_text,
                'segments': transcript_list
            }
        except TranscriptsDisabled:
            raise Exception(f"❌ Transcripts are disabled for video: {video_id}")
        except NoTranscriptFound:
            raise Exception(f"❌ No transcript found for video: {video_id}")
        except Exception as e:
            raise Exception(f"❌ Error fetching transcript: {str(e)}")
    
    def fetch_multiple(self, video_ids: list) -> list:
        """Fetch transcripts for multiple videos"""
        transcripts = []
        for i, video_id in enumerate(video_ids, 1):
            print(f"[{i}/{len(video_ids)}] Fetching: {video_id}")
            try:
                transcript = self.fetch_transcript(video_id)
                transcripts.append(transcript)
                print(f"  ✅ Success! Got {len(transcript['transcript'])} characters")
            except Exception as e:
                print(f"  {str(e)}")
        return transcripts

print("✅ Transcript fetcher ready")

## Step 5: Add YouTube Videos

Enter video IDs or URLs (one per line)

In [ ]:
# Enter your YouTube video IDs or URLs here
video_ids = [
    "jNQXAC9IVRw",  # Example: Me at the zoo
    # Add more video IDs here
]

# Or input manually
manual_input = input("Enter video IDs (comma-separated) or press Enter to use default: ").strip()
if manual_input:
    video_ids = [v.strip() for v in manual_input.split(',')]

print(f"\n📥 Fetching {len(video_ids)} video(s)...\n")

fetcher = YouTubeTranscriptFetcher()
transcripts = fetcher.fetch_multiple(video_ids)

print(f"\n✅ Successfully fetched {len(transcripts)} transcript(s)!")

## Step 6: Create Text Chunks

In [ ]:
# Create LangChain documents
documents = []
for transcript in transcripts:
    doc = Document(
        page_content=transcript['transcript'],
        metadata={
            'video_id': transcript['video_id'],
            'url': f"https://www.youtube.com/watch?v={transcript['video_id']}"
        }
    )
    documents.append(doc)

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

chunks = text_splitter.split_documents(documents)

print(f"✅ Created {len(chunks)} text chunks")

## Step 7: Create Vector Database

In [ ]:
print("🔄 Creating embeddings... (this may take a minute)")

# Create embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create vector store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print(f"✅ Vector database created with {len(chunks)} chunks!")

## Step 8: Create RAG Chatbot

In [ ]:
# Create ChatGPT instance
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)

# Create retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True
)

print("✅ RAG Chatbot ready!")

## Step 9: Chat Function

In [ ]:
def chat(question: str):
    """Ask a question about your videos"""
    print(f"\n❓ Question: {question}\n")
    
    result = qa_chain({"query": question})
    
    print(f"💬 Answer: {result['result']}\n")
    
    # Show sources
    if result.get('source_documents'):
        print("📚 Sources:")
        for i, doc in enumerate(result['source_documents'], 1):
            video_id = doc.metadata.get('video_id', 'Unknown')
            print(f"  [{i}] Video: {video_id}")
            print(f"      URL: https://www.youtube.com/watch?v={video_id}")
    
    return result['result']

print("✅ Chat function ready! Use: chat('your question here')")

## Step 10: Start Chatting!

Now you can ask questions about your videos:

In [ ]:
# Example questions
chat("What is this video about?")

In [ ]:
# Ask your own question
question = input("Your question: ")
chat(question)

## Step 11: Interactive UI with Gradio (Optional)

In [ ]:
import gradio as gr

def gradio_chat(message, history):
    """Gradio chat interface"""
    result = qa_chain({"query": message})
    
    # Build response with sources
    response = result['result']
    
    if result.get('source_documents'):
        response += "\n\n📚 **Sources:**\n"
        for i, doc in enumerate(result['source_documents'][:2], 1):
            video_id = doc.metadata.get('video_id', 'Unknown')
            response += f"- [Video {video_id}](https://www.youtube.com/watch?v={video_id})\n"
    
    return response

# Create Gradio interface
demo = gr.ChatInterface(
    fn=gradio_chat,
    title="🎥 YouTube RAG Chatbot",
    description=f"Ask questions about {len(transcripts)} YouTube video(s)",
    examples=[
        "What is the main topic?",
        "Summarize the key points",
        "What are the important details?"
    ]
)

# Launch with public link
demo.launch(share=True)

## 🎉 You're Done!

### Usage:
1. Use `chat("your question")` in any cell
2. Or use the Gradio UI above

### Add More Videos:
Run Step 5 again with new video IDs, then re-run Steps 6-8

### Tips:
- Use videos with captions enabled
- Try educational content, tutorials, or lectures
- The more videos you add, the more knowledge the chatbot has!